In [2]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import BitcoinQuerier, BitcoinProcessor, BitcoinPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')


In [4]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

query = mongodb_query_ops.get_recent_blocks('Bitcoin')
data = query[0]['raw_block_data']

2025-01-10 01:26:43,862 - INFO - Connected to MongoDB database: blockchain
2025-01-10 01:26:43,883 - INFO - Indexes created for collection: Solana
2025-01-10 01:26:43,885 - INFO - Indexes created for collection: Bitcoin
2025-01-10 01:26:43,887 - INFO - Indexes created for collection: Ethereum
2025-01-10 01:26:43,889 - INFO - Indexes created for collection: BNB
2025-01-10 01:26:43,891 - INFO - Indexes created for collection: XRP
2025-01-10 01:26:43,907 - INFO - Retrieved 4 most recent blocks from the Bitcoin collection.


In [13]:
len(data['tx'])

4360

In [12]:
data['tx'][0]

{'txid': '70ee20c7cc6789667d79e7398268324313d0ccadf06d825fb89bc260e8fb5380',
 'hash': '0825ed681ba82d61ba8f2c032048910ba8087a89d83618caecfabcd0a1f0ead1',
 'version': 1,
 'size': 421,
 'vsize': 394,
 'weight': 1576,
 'locktime': 0,
 'vin': [{'coinbase': '036e670d194d696e656420627920416e74506f6f6c206c01c002da0a0ce7fabe6d6de8217c34784e68bb89f1ee51e01d7fb27eb9a90d8c5a58aa593bf444ed2952461000000000000000cbd100002902010000000000',
   'txinwitness': ['0000000000000000000000000000000000000000000000000000000000000000'],
   'sequence': 4294967295}],
 'vout': [{'value': 5.46e-06,
   'n': 0,
   'scriptPubKey': {'asm': 'OP_HASH160 42402a28dd61f2718a4b27ae72a4791d5bbdade7 OP_EQUAL',
    'desc': 'addr(37jKPSmbEGwgfacCr2nayn1wTaqMAbA94Z)#avhxp88d',
    'hex': 'a91442402a28dd61f2718a4b27ae72a4791d5bbdade787',
    'address': '37jKPSmbEGwgfacCr2nayn1wTaqMAbA94Z',
    'type': 'scripthash'}},
  {'value': 3.16522382,
   'n': 1,
   'scriptPubKey': {'asm': 'OP_HASH160 5249bdf2c131d43995cff42e8feee293f79297a8 

In [10]:
data['tx'][1]

{'txid': 'df9e87d4c18496db909cd6c2a310dff617e4e672252755a12b8aa9838528defc',
 'hash': 'ece7fd567d6de6bf8db485573956cb6f40d8ad6b80dee6c7e3950c5de196910e',
 'version': 1,
 'size': 392,
 'vsize': 201,
 'weight': 803,
 'locktime': 0,
 'vin': [{'txid': 'dbb204dfa170d110ff06a672bda9f60fccabbb88c761289dce9e25055492da33',
   'vout': 1,
   'scriptSig': {'asm': '', 'hex': ''},
   'txinwitness': ['',
    '3045022100e46190f1eaeaef2f68bf46e7c10aa77ed21897d13a16dbe9a39491acab0dc54d022075616e3f775e3d77c4a764489b6bb7bb24e860f4d1e73c294ea7e015ec9f133d01',
    '30440220020b062bb1f5af34ad97046dcff79496ee213b5d1294985d7b4d1b0aa6c4b72f02203261e9dde769ae61b15eff9f8a0d97b5e7dc41bb8153a284da9cfb15dcfb35b301',
    '5221025adaa68725011aa2fe3dccf6cfd36ac7937a477d0d716684f6c7cc1b489ab1b62103eb4755698f928baae215891562e7f955c3ede634c5ce39d3839c3ea6213a04f621034d0e66b77cbc6b2a7f3f9bef3edf15459763aa05878ff3b634841b9f065fcd0e53ae'],
   'sequence': 4294967293}],
 'vout': [{'value': 0.0328,
   'n': 0,
   'scriptPubKey':

In [11]:
for tx in data['tx']:
    for vin in tx['vin']:
        if vin.get('value',0) != 0:
            print(tx)
